# 0. Preliminaries

In [1]:
import json
import random
import os
from PIL import Image
from itertools import product
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from collections import Counter

from domain import SYM2PROG, Program, NULL_VALUE
import sys
from helper import *

# 1. Split handwritten symbols

In [2]:
# split train and test set of math symbols
random.seed(777)
terminals = ['+', '-', 'times', 'div', '(', ')', '!'] + list(map(str, list(range(10)))) + ['alpha', 'beta', 'gamma', 'theta', 'phi'] + list('abcdexyz')
sym_train_set = {}
sym_val_set = {}
sym_test_set = {}
for tok in terminals:
    imgs = os.listdir(symbol_images_dir + tok)
    random.shuffle(imgs)
    n_train = int(len(imgs) * 0.75)
    n_val = int(len(imgs) * 0.05)
    sym_train_set[tok] = sorted(imgs[:n_train])
    sym_val_set[tok] = sorted(imgs[n_train:n_train+n_val])
    sym_test_set[tok] = sorted(imgs[n_train+n_val:])
    print(tok, len(imgs))
json.dump(sym_train_set, open('sym_train.json', 'w'))
json.dump(sym_val_set, open('sym_val.json', 'w'))
json.dump(sym_test_set, open('sym_test.json', 'w'))

+ 5443
- 6022
times 600
div 157
( 3986
) 3978
! 224
0 1810
1 6327
2 6210
3 2469
4 1641
5 1008
6 812
7 753
8 731
9 742
alpha 383
beta 295
gamma 90
theta 543
phi 83
a 2724
b 1830
c 1194
d 1062
e 616
x 5333
y 1895
z 1075


# 2. Generate expressions

In [3]:
random.seed(777)
split2sym = {}
for split in splits:
    sym_set = {k:Iterator(v) for k,v in json.load(open('sym_%s.json'%split)).items()}
    split2sym[split] = sym_set

In [4]:
random.seed(12306)
max_op = 20
max_op_train = 10
max_value_train = 100
max_value_test = 10000
n_train = 100000
n_test = 1000
n_val = 100
res_max_ratio = 0.05

train_exprs = []
val_exprs = []
test_exprs = []

for n_op in range(max_op + 1):
    if n_op > max_op_train:
        n_train = 0
    expressions = generate_expression(n_op, n_train + n_val + n_test, max_value=max_value_train, res_max_ratio=res_max_ratio)
    if len(expressions) < n_train:
        print('there are not enough expressions for train set. Repeat them.')
        if n_op == 0:
            expressions = expressions * (20 * n_train // 1000)
        elif n_op == 1:
            expressions = expressions * (2 * n_train // 1000)
        else:
            expressions = expressions * (n_train // len(expressions) + 1)
            expressions = expressions[:n_train]
    
    train_exprs_i = []
    val_exprs_i = []
    test_exprs_i = []

    # split 'I'
    if n_train > 0:
        train_exprs_i.extend(expressions[:n_train])
        val_exprs_i.extend(expressions[:len(train_exprs_i) * n_val // n_train])
        test_exprs_i.extend(expressions[:len(train_exprs_i) * n_test // n_train])
    
    # split 'SS' or 'LS'
    val_exprs_i.extend(expressions[n_train:n_train+n_val])
    test_exprs_i.extend(expressions[n_train+n_val:])
    
    # split 'SL' or 'LL'
    expressions = generate_expression(n_op, n_val + n_test, min_value=max_value_train+1, max_value=max_value_test, res_max_ratio=res_max_ratio)
    val_exprs_i.extend(expressions[:n_val])
    test_exprs_i.extend(expressions[n_val:n_val+n_test])

    print(n_op, len(train_exprs_i), len(val_exprs_i), len(test_exprs_i))
    train_exprs.append(train_exprs_i)
    val_exprs.append(val_exprs_i)
    test_exprs.append(test_exprs_i)

    
split2exprs = {'train': train_exprs, 'val': val_exprs, 'test': test_exprs}
print([(k, sum([len(x) for x in v])) for k, v in split2exprs.items()])

there are not enough expressions for train set. Repeat them.
0 20000 20 200
there are not enough expressions for train set. Repeat them.
1 78000 78 780
there are not enough expressions for train set. Repeat them.


  1%|▍                                 | 1408/101100 [00:00<00:07, 14066.83it/s]

2 100000 200 1640


  1%|▎                                 | 1073/101100 [00:00<00:09, 10723.11it/s]

3 100000 300 3000


...

18 0 200 2000


  8%|███▏                                    | 89/1100 [00:00<00:01, 885.60it/s]

19 0 200 2000


100%|██████████████████████████████████████| 1100/1100 [00:01<00:00, 565.41it/s]


20 0 200 2000
[('train', 998000), ('val', 4698), ('test', 46620)]


In [5]:
split2dataset = {}
for split in splits:
    sym_set = split2sym[split]
    exprs = split2exprs[split]
    dataset = []
    idx = 0
    for n_op, expr_list in enumerate(exprs):
        for e in expr_list:
            if split in ['val', 'test']:
                if e[0] in train_exprs:
                    evaluation = 'I'
                else:
                    max_value = max([x for x in e[3] if x is not None])
                    evaluation = 'S' if n_op <= max_op_train else 'L'
                    evaluation += 'S' if max_value <= max_value_train else 'L'
            
            img_paths = generate_img_paths(e[0], sym_set)
            sample = {'id': '%s_%08d'%(split, idx), 'img_paths':img_paths, 
                  'expr': e[0], 'head': e[1], 'res': e[2], 'res_all': e[3]}
            if split in ['val', 'test']:
                sample['eval'] = evaluation
            idx += 1
            dataset.append(sample)
    split2dataset[split] = dataset
    if split == 'train':
        train_exprs = {x['expr'] for x in dataset}
    print(split, len(dataset))
for split in splits:
    json.dump(split2dataset[split], open('expr_%s.json'%split, 'w'))

train 998000
val 4698
test 46620


In [6]:
counts = sorted(Counter([x['eval'] for x in split2dataset['test']]).items())
total_count = sum([c for _, c in counts])
counts = [(k, v, round(v/total_count*100, 2)) for k, v in counts]
print(counts)

[('I', 9980, 21.41), ('LL', 10000, 21.45), ('LS', 10000, 21.45), ('SL', 8640, 18.53), ('SS', 8000, 17.16)]
